In [4]:
import pandas as pd
import plotly.express as px
import numpy as np

# Load the data
df = pd.read_csv('gpu-pricing.csv')
df.head()

# Create a numerical representation of the NVIDIA GPU Name column
df['NVIDIA GPU Name Numeric'] = pd.factorize(df['NVIDIA GPU Name'])[0]

# Increase the jitter range to reduce overlap
df['NVIDIA GPU Name Jittered'] = df['NVIDIA GPU Name Numeric'] + np.random.uniform(-0.5, 0.5, size=len(df))

# Define a custom color sequence with darker, high-contrast colors
custom_colors = ['#ffc300', '#000000', '#ff5733', '#c70039', '#900c3f', '#581845', '#1c1c1c', '#2e2e2e']

# Define specific colors for AWS, Azure, OLA, and Lambda
color_map = {
    'AWS': '#FFC000',  # Yellow
    'Azure': '#0000FF',  # Blue
    'OLA': '#000000',  # Bright Red
    'Lambda': '#FF0000'  # Black
}

# Create interactive scatter plot with jitter to avoid overlapping bubbles
fig = px.scatter(df, x='NVIDIA GPU Name Jittered', y='INR PER HOUR PER GPU RAM', color='Provider',
                 size='INR PER HOUR PER GPU RAM',
                 title='NVIDIA GPU Pricing by Provider', labels={'INR PER HOUR PER GPU RAM': 'INR Per Hour per GPU RAM'},
                 hover_data={'Provider': True, 'NVIDIA GPU Name': True, 'INR PER HOUR PER GPU RAM': True, 'NVIDIA GPU Name Jittered': False},
                 color_discrete_sequence=custom_colors,
                 color_discrete_map=color_map)

# Update layout to remove grid
fig.update_layout(
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False)
)

# Add annotations for unique GPU names below the x-axis, stacked vertically
unique_gpus = df.drop_duplicates(subset=['NVIDIA GPU Name'])
annotations = []
y_offset = -0.1
for i, row in unique_gpus.iterrows():
    annotations.append(dict(
        x=row['NVIDIA GPU Name Jittered'],
        y=y_offset,  # Position below the x-axis, stacked vertically
        text=row['NVIDIA GPU Name'],
        showarrow=False,
        xanchor='center',
        yanchor='top',
        font=dict(size=10)
    ))
    y_offset -= 0.05  # Decrease y_offset to stack names vertically

fig.update_layout(annotations=annotations)

# Save the plot as an interactive HTML file
fig.write_html('gpu_pricing.html')

# Show the plot
fig.show()